# 데이터 전처리

## 데이터 로드

In [1]:
import pickle, os
path = os.getenv("HOME") + '/project_data/naver_movie_review.pickle'

In [2]:
import pandas as pd
data = pd.read_pickle(path)
data.head()

,number,title,score,review,auther
0,17238977,시네마 천국,10,야외상영할 때 엘레나가 갑작스레 와서 키스할 때 진짜 너무 설레였다... 사랑하고 ...,wusl****
1,17238637,시네마 천국,10,올타임 넘버원 모든 면에서 최고의 영화~,park****
2,17237730,시네마 천국,9,토토의 영화관은 사라져도 토토와 알프레도의 추억은 계속해서 상영기에서 돌아간다.,kjes****
3,17236527,시네마 천국,5,이거보다 더 오래된 영화여도 충분히 감동 깊게 재밌게 봤던 영화도 많았다. 대부를 ...,mrki****
4,17234710,시네마 천국,10,,wldu****


## 결측치 제거

In [3]:
len(data)

8114820

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8114820 entries, 0 to 8114819
Data columns (total 5 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   number  object
 1   title   object
 2   score   object
 3   review  object
 4   auther  object
dtypes: object(5)
memory usage: 309.6+ MB


In [5]:
data = data[data['review'] != '']

In [6]:
len(data)

8062953

## 매트릭스 생성

### 유저 인코딩

In [7]:
data.nunique()

number    8062952
title        7212
score          10
review    7669642
auther     385180
dtype: int64

In [8]:
user_to_idx = {u:idx for idx, u in enumerate(data['auther'].unique())}
len(user_to_idx)

385180

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(data['auther'].unique())

LabelEncoder()

In [10]:
le.classes_

array(['0-is****', '0-na****', '00--****', ..., 'zzzx****', 'zzzy****',
       'zzzz****'], dtype=object)

In [11]:
%timeit data['user_idx'] = le.transform(data.auther)

1.89 s ± 10 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
data.head()

,number,title,score,review,auther,user_idx
0,17238977,시네마 천국,10,야외상영할 때 엘레나가 갑작스레 와서 키스할 때 진짜 너무 설레였다... 사랑하고 ...,wusl****,353137
1,17238637,시네마 천국,10,올타임 넘버원 모든 면에서 최고의 영화~,park****,259395
2,17237730,시네마 천국,9,토토의 영화관은 사라져도 토토와 알프레도의 추억은 계속해서 상영기에서 돌아간다.,kjes****,192403
3,17236527,시네마 천국,5,이거보다 더 오래된 영화여도 충분히 감동 깊게 재밌게 봤던 영화도 많았다. 대부를 ...,mrki****,229566
5,17234247,시네마 천국,10,어딘가 마음이 꿈틀대는 영화.,s2fr****,292160


In [14]:
data['auther'].nunique() == data['user_idx'].nunique()

True

### 영화인코딩
- apply보다 LabelEncoder가 빠름
- 더 빠른 방법으로 추후 변경예정

In [15]:
movie_to_idx = {m:idx for idx, m in enumerate(data['title'].unique())}
len(movie_to_idx)

7212

In [16]:
%timeit data['movie_index'] = data.apply(lambda x : movie_to_idx[x['title']], axis = 1)

37.9 s ± 366 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
le2 = LabelEncoder()
le2.fit(data['title'].unique())

LabelEncoder()

In [19]:
%timeit data['movie_index2'] = le2.transform(data.title)

649 ms ± 2.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 매트릭스

In [20]:
data.head()

,number,title,score,review,auther,user_idx,movie_index,movie_index2
0,17238977,시네마 천국,10,야외상영할 때 엘레나가 갑작스레 와서 키스할 때 진짜 너무 설레였다... 사랑하고 ...,wusl****,353137,0,4095
1,17238637,시네마 천국,10,올타임 넘버원 모든 면에서 최고의 영화~,park****,259395,0,4095
2,17237730,시네마 천국,9,토토의 영화관은 사라져도 토토와 알프레도의 추억은 계속해서 상영기에서 돌아간다.,kjes****,192403,0,4095
3,17236527,시네마 천국,5,이거보다 더 오래된 영화여도 충분히 감동 깊게 재밌게 봤던 영화도 많았다. 대부를 ...,mrki****,229566,0,4095
5,17234247,시네마 천국,10,어딘가 마음이 꿈틀대는 영화.,s2fr****,292160,0,4095


In [24]:
nu, ni = data['auther'].nunique(), data['title'].nunique()
print(nu, ni)

385180 7212


In [30]:
import numpy as np
MF = np.zeros([nu, ni], dtype = np.dtype('f4'))
MF.shape

(385180, 7212)

In [33]:
def fill_mf(user_idx, movie_idx, score):
    MF[user_idx][movie_idx] = int(score)
fill_mf(data['user_idx'].values, data['movie_index2'].values, data['score'].values)

TypeError: only size-1 arrays can be converted to Python scalars